In [ ]:
# ============================================
# Arabic NER with BERT (WikiANN)
# ============================================

!pip install -q datasets transformers seqeval accelerate

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from seqeval.metrics import f1_score, classification_report
import numpy as np
import torch

# -----------------------------
# 1. Load WikiANN Arabic
# -----------------------------
dataset = load_dataset("wikiann", "ar")

train_ds = dataset["train"]
val_ds   = dataset["validation"]
test_ds  = dataset["test"]

print("Train:", len(train_ds))
print("Val:  ", len(val_ds))
print("Test: ", len(test_ds))

label_names = train_ds.features["ner_tags"].feature.names
print("Labels:", label_names)


# -----------------------------
# 2. Convert to HF Dataset format
# -----------------------------
def convert(ds):
    sentences = []
    for ex in ds:
        tokens = ex["tokens"]
        labels = [label_names[i] for i in ex["ner_tags"]]
        sentences.append({"tokens": tokens, "ner_tags": labels})
    return sentences

train_data = convert(train_ds)
val_data   = convert(val_ds)
test_data  = convert(test_ds)

print("Samples loaded.")


# -----------------------------
# 3. Label mapping
# -----------------------------
unique_labels = sorted(list(set(label for ex in train_data for label in ex["ner_tags"])))
label_to_id = {label: i for i, label in enumerate(unique_labels)}
id_to_label = {i: label for label, i in label_to_id.items()}

print("Label mapping:", label_to_id)


# HF Dataset
from datasets import Dataset, DatasetDict

train_hf = Dataset.from_list(train_data)
val_hf   = Dataset.from_list(val_data)
test_hf  = Dataset.from_list(test_data)

raw_datasets = DatasetDict({
    "train": train_hf,
    "validation": val_hf,
    "test": test_hf
})

# -----------------------------
# 4. Tokenize + Align labels
# -----------------------------
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_and_align(example):
    tokenized = tokenizer(
        example["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=256,
    )

    word_ids = tokenized.word_ids()
    labels = []

    for word_id in word_ids:
        if word_id is None:
            labels.append(-100)
        else:
            labels.append(label_to_id[example["ner_tags"][word_id]])

    tokenized["labels"] = labels
    return tokenized

tokenized_datasets = raw_datasets.map(
    tokenize_and_align,
    batched=False,
    remove_columns=["tokens", "ner_tags"]
)

print("Tokenization complete.")


# -----------------------------
# 5. Load BERT for Token Classification
# -----------------------------
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(unique_labels),
    id2label=id_to_label,
    label2id=label_to_id
)


# -----------------------------
# 6. TrainingArguments (compatible)
# -----------------------------
training_args = TrainingArguments(
    output_dir="./bert_ner_arabic",
    do_train=True,
    do_eval=True,
    learning_rate=5e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=100,
    save_steps=500,
    eval_steps=500,
    weight_decay=0.01,
)


# -----------------------------
# 7. Trainer
# -----------------------------
def compute_metrics(p):
    preds = p.predictions if hasattr(p, "predictions") else p[0]
    labels = p.label_ids if hasattr(p, "label_ids") else p[1]

    preds = np.argmax(preds, axis=-1)

    true_preds = []
    true_labels = []

    for pred_seq, label_seq in zip(preds, labels):
        for p_id, l_id in zip(pred_seq, label_seq):
            if l_id == -100:
                continue
            true_labels.append(id_to_label[l_id])
            true_preds.append(id_to_label[p_id])

    f1 = f1_score([true_labels], [true_preds])  # simplified for batch
    return {"f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

trainer.train()


# -----------------------------
# 8. Evaluate on Test Set
# -----------------------------
output = trainer.predict(tokenized_datasets["test"])
preds = np.argmax(output.predictions, axis=-1)
labels = output.label_ids

y_true = []
y_pred = []

for pred_seq, label_seq in zip(preds, labels):
    true_seq = []
    pred_tag = []
    for p, l in zip(pred_seq, label_seq):
        if l == -100:
            continue
        true_seq.append(id_to_label[l])
        pred_tag.append(id_to_label[p])
    y_true.append(true_seq)
    y_pred.append(pred_tag)

print("\n=== Test Set Report (BERT NER) ===")
print("F1:", f1_score(y_true, y_pred))
print(classification_report(y_true, y_pred))


# -----------------------------
# 9. Predict on custom sentence
# -----------------------------
def predict_sentence_bert(tokens):
    encoding = tokenizer(
        tokens,
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=50,
    ).to(model.device)

    with torch.no_grad():
        outputs = model(**encoding)

    preds = outputs.logits.argmax(-1)[0]
    word_ids = encoding.word_ids()

    result = []
    used = set()
    for p_id, w_id in zip(preds.tolist(), word_ids):
        if w_id is None:
            continue
        if w_id in used:
            continue
        used.add(w_id)
        result.append((tokens[w_id], id_to_label[p_id]))

    return result


example = ["زار", "الملك", "العاصمة", "عمان", "أمس", "."]
print("\nCustom example:")
print(predict_sentence_bert(example))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

ar/validation-00000-of-00001.parquet:   0%|          | 0.00/643k [00:00<?, ?B/s]

ar/test-00000-of-00001.parquet:   0%|          | 0.00/648k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-4237898834.py:158: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hamadourabah1975 (hamadourabah1975-i-stinye-niversitesi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,0.711000
200,0.411700
300,0.378000
400,0.369800
500,0.317700
600,0.322400
700,0.328800
800,0.300100
900,0.291000
1000,0.299800
